In [ ]:
import mercury as mr
import requests
import time
import base64

In [ ]:
#https://github.com/smurphy13IIT/Digital_Signage_Builder.git
owner = "smurphy13IIT"
repo_name = "Digital_Signage_Builder"


In [ ]:
#fork_repository(owner, repo_name, token)
def fork_repository(owner, repo_name, token):
    url = f"https://api.github.com/repos/{owner}/{repo_name}/forks"
    headers = {
        "Authorization": f"bearer {token}",
        "Accept": "application/vnd.github.v3+json"
    }
    response = requests.post(url, headers=headers)
    print(response)

def write_string_to_file(my_repo, repo_name, file_path, slideshow_name, token, file_content):
    url = f"https://api.github.com/repos/{my_repo}/{repo_name}/contents/sites/{slideshow_name}/{file_path}"
    headers = {
        "Authorization": f"token {token}",
        "Accept": "application/vnd.github.v3+json",
        "X-GitHub-Api-Version": "2022-11-28"
    }

    # Encode file content as base64
    content_bytes = file_content.encode('utf-8')
    content_base64 = base64.b64encode(content_bytes).decode('utf-8')

    data = {
        "message": "Write string to file",
        "content": content_base64
    }

    try:
        response = requests.put(url, headers=headers, json=data)
        response.raise_for_status()
        #print("String written to file successfully!")

    except requests.HTTPError as e:
        print(f"Error writing string to file: {e}")

In [ ]:
app = mr.App(title="Git Connection", description="App for forking the Digital Signage Builder Github repository.")
username_widget = mr.Text(value="username", label="Provide your Github Username:", rows=1)
token_widget = mr.Text(value="token", label="Provide your Github Personal Token:", rows=1)
action_select = mr.Select(value= "Update Slides", choices= ["Update Slides", "Create New Slides App"], label= "Select an action, then click 'Run':")

In [ ]:
if action_select.value == "Create New Slides App":
    button = mr.Button(label="Create Signage Repo", style="primary")
    token = token_widget.value
    username = username_widget.value
    fork_repository(owner, repo_name, token)
    mr.Markdown(text="""#New Signage App Successfully Created!"
    """)

elif action_select.value == "Update Slides":
    slideshow_input = mr.Text(value="Slideshow Name", label="Give your slideshow a name:")
    slideshow_name = slideshow_input.value
    token = token_widget.value
    start_update = mr.Button(label="Begin Slides Update", style="primary")
    commit_message = "create file via PyGitHub"
    file_content = "Hello World!"
    filename = f"{slideshow_name}.txt"
    file_path = f"{slideshow_name}"

    if start_update.clicked:
        write_string_to_file(username_widget.value, repo_name, file_path, slideshow_name, token, file_content)
        mr.Markdown(text="""#Slides Update Complete!
        """)

# Digital Signage Builder
## IIT Galvin Library
#### Step 1: Create a Github account by <a href="https://docs.github.com/en/get-started/start-your-journey/creating-an-account-on-github">following these instructions</a>.
#### Step 2: Create a Personal Access Token <a href="https://docs.github.com/en/enterprise-server@3.9/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens">following these steps</a>.
#### NOTE: Your Github Personal Access Token is like a password. While this script will not save your token after you leave, you should treat it securely like any other password.
#### Step 3: Type your Github username in the "username" text box, then hit "enter".
#### Step 4: Paste your Github Personal Access Token in the "token" text box, then hit "enter".
#### Step 5: Click "Create Signage Repo" to copy your own version of the Digital Signage Builder installation.

NEED TO ADD:

- Method of uploading slide images for the slideshow
- Code to check existing slides folder and remove any that aren't submitted to this app
- Code to set up Github Page for the user's Digital Signage Builder fork